## Reading Data from the STAC API

In [8]:
MISSING_IMPORTS = False
ASSET = "preview" # thumbnail
PRINT_STATS = False
SAVE_IM = True
LAST_SAVED_IM = 0 # Change if you have images for not overwriting

In [9]:
if MISSING_IMPORTS:
    import sys
    !{sys.executable} -m pip install pystac-client
    !{sys.executable} -m pip install planetary-computer

## Åpne katalog

In [10]:
from pystac_client import Client

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
collections = catalog.get_children()

## Definer område og søk etter bilder

In [11]:
# Coral reef
"""area_of_interest = {
  "type": "Polygon",
  "coordinates": [FILL IN HERE]
}

# Coral reefs - Australia original
[
      [148.23531179359935, -20.54493998596469],
      [150.34774273804175, -20.54493998596469],
      [150.34774273804175, -19.109271683495322],
      [148.23531179359935, -19.109271683495322],
      [148.23531179359935, -20.54493998596469],
]
date_range = "2015-01-01/2022-02-28" # Train

# CORAL REEFS - FIJI
[
      [-179.92271497924932, -16.875196086915338],
      [-178.45204169243533, -16.875196086915338],
      [-178.45204169243533, -16.010480325382275],
      [-179.92271497924932, -16.010480325382275],
      [-179.92271497924932, -16.875196086915338]
]
date_range = "2021-01-01/2022-02-28" # Test

# Mangroves
[
      [105.3529771100221, -2.414913966588884],
      [105.46295194928183, -2.414913966588884],
      [105.46295194928183, -2.3539581262169946],
      [105.3529771100221, -2.3539581262169946],
      [105.3529771100221, -2.414913966588884]
]
[
      [104.46575834438039, -1.9812916699269891],
      [104.74276457631794, -1.9812916699269891],
      [104.74276457631794, -1.827707081113374],
      [104.46575834438039, -1.827707081113374],
      [104.46575834438039, -1.9812916699269891]
]
"""

area_of_interest = {
  "type": "Polygon",
  "coordinates": [
    [
       [-179.92271497924932, -16.875196086915338],
      [-178.45204169243533, -16.875196086915338],
      [-178.45204169243533, -16.010480325382275],
      [-179.92271497924932, -16.010480325382275],
      [-179.92271497924932, -16.875196086915338]

    ]
  ]
}

date_range = "2021-01-01/2022-02-28"
search = catalog.search(filter_lang="cql2-json", filter={
  "op": "and",
  "args": [
    {"op": "s_intersects", "args": [{"property": "geometry"}, area_of_interest]},
    {"op": "anyinteracts", "args": [{"property": "datetime"}, date_range]},
    {"op": "=", "args": [{"property": "collection"}, "sentinel-2-l2a"]},
    {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 10]}
  ]
})

items = list(search.get_items())

## Hent inn og signer hvert bilde

In [12]:
import planetary_computer as pc

images = [item.assets[ASSET] for item in items]
signed_images = [pc.sign(im.href) for im in images]


## Print stats

In [13]:

if PRINT_STATS:
    # Print collections from planetary hub
    for collection in collections:
        print(f"{collection.id} - {collection.title}")

    # Print all image uri's
    # Can access one of them by the URI:
    # https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-2-l2a/items/[item.id]
    for item in items:
        print(f"{item.id}: {item.datetime}")

    # Print properties of item
    print(items[0].properties.keys())

    # Print assets of item
    print(items[0].assets.keys())

## Vis/Lagre bilder

In [14]:
from PIL import Image
from urllib.request import urlopen

if SAVE_IM:
    for i, image in enumerate(signed_images):
        Image.open(urlopen(image)).save(f"../data_preparation/images/test2022/im{i+LAST_SAVED_IM}.jpg")
else:
    Image.open(urlopen(signed_images[0]))